# Climate Services workshop

This notebook and workshop was developed for the [ICEM 2023 conference](https://www.wemcouncil.org/wp/icem-2023-speakers-program), presented at the [Climate Services workshop](https://linktr.ee/ICEM23ClimateServices) using [`jupyter RISE`](https://rise.readthedocs.io/en/stable), and intended to work on https://mybinder.org.

*Advanced `python` users: see the github repo README for more instructions with setup and installation if running on your own device.*

In [ ]:
%%capture
# ensure that all package requirements are met
!pip install pandas xarray rioxarray matplotlib rasterio

In [ ]:
# import relevant libraries and functions not built-in to python

# numpy is used for working with arrays, and has many other uses
import numpy as np

# pandas is a really nice tool for working with 2D arrays and reading CSV files
import pandas as pd

# xarray has more features than pandas, and can handle
# more complex data structures and types (including netcdf)
import xarray as xr
import rioxarray as rx

In [ ]:
# hide warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# plotting and styling

# matplotlib is a general purpose plotting backend
import matplotlib.pyplot as plt

# plotting large datasets
import rasterio
from rasterio.plot import show

# [optional] use the mplcyberpunk theme, and pass if module not found
try:
    # theme can be installed with: pip install mplcyberpunk
    import mplcyberpunk
    plt.style.use("cyberpunk")
except:
    pass

# change default plot size
plt.rcParams["figure.figsize"] = (12, 3.8)

<center><h1><a id="top">Workshop on Weather and Climate Services</a></h1></center>

<center>Using weather and climate data to support decision-making in the energy sector</center>

<center>
    <b>icem 2023</b> 7th International Conference Energy & Meteorology:
    <br>
    <i>Towards Climate-Resilient Energy Systems</i>
</center>

**Convenors: James Fallon, Jake Badger and Justin Sharp (remote)** 

<div style="width:14%;float: left">
<a href="https://linktr.ee/ICEM23ClimateServices"><img src="github_qr.png"></a>
</div>
<br><br>&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://linktr.ee/ICEM23ClimateServices">https://linktr.ee/ICEM23ClimateServices</a>


## Programme

*Tuesday 27th June 16:15-18:00 CEST*  

**Panellists: Hiba Omrani, Falguni Patadia, Frank Kaspar, Gabriel Perez**

| Time | Activity | Speaker |
| --- | --- | --- |
| 16:15 | Introduction | James |
| 16:20 | Overview of Climate Services | Jake, Panellists |
| 16:40 | Interactive Session: Climate Services Notebook | James |
| 17:20 | Discussion: the "Next Generation" of Climate Services | Panellists |
| 17:55 | Closing remarks | Justin |

## Introduction



### What is a climate service?

*A decision aide derived from climate information*

Example slides and schematics here: <https://www.wemcouncil.org/TALKS/EEA_Troccoli_Copenhagen_Sep2018.pdf>

WMO Definition: <https://public.wmo.int/en/bulletin/what-do-we-mean-climate-services>

#### Definition of Climate Services adopted by the ScienceDirect Climate Services journal (open access)

"the transformation of climate-related data - together with other relevant information - into customized products such as projections, forecasts, information, trends, economic analysis, assessments (including technology assessment), counselling on best practices development and evaluation of solutions and any other services in relation to climate that may be use for the society at large."

source: European Commission's Roadmap for Climate Services (2015).

### What is a climate service used for?

### What we will cover today?

* introduction from expert panellists
* how to **access and use** some climate services with python notebooks
* things to be aware of
* discussion on the next generation of weather & climate services

## Overview of Live Climate Services

**Panellists:**

* Falguni Patadia (NASA POWER)
* Hiba Omrani (FOCUS-AFRICA)
* Gabriel Perez (Meteo IA)
* Frank Kaspar (DWD)

<center><img src="panellist slides PDF/ICEM 2023 Falguni Patadia NASA POWER.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Hiba OMRANI CS 1.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Hiba OMRANI CS 2.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Hiba OMRANI CS 3.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Hiba OMRANI CS 4.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Hiba OMRANI CS 5.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Gabriel Perez MeteoIA 1.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Gabriel Perez MeteoIA 2.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Gabriel Perez MeteoIA 3.jpg" width="100%"></center>

<center><img src="panellist slides PDF/ICEM 2023 Gabriel Perez MeteoIA 4.jpg" width="100%"></center>

## Interactive Session Part 1: Working with Meteorological Data

We will show how to read in wind data - but a similar process can be considered for solar irradiance, temperature, and other energy-relevant variables

### Downloading data...

In this example, we use the **NASA POWER DAVe** tool to extract **1 month of hourly 50m wind speed data** from **Hudson Bay**

<center><img src="screenshots/NASA_POWER_demo_2.png" width="80%"></center>

### Read in a reanalysis dataset

Now that we have downloaded our data, we use `xarray` to open the dataset

In [ ]:
# read in the dataset
wind50m_reanalysis = xr.open_dataset("datasets/reanalysis/POWER_Point_Hourly_20220129_20230128_039d48N_073d59W_LST.nc")["WS50M"]

In [ ]:
# dataset overview
wind50m_reanalysis

### Observations dataset

In [ ]:
def read_csv_lidar(fpath: str, field: str, na_value: float=9990.) -> pd.DataFrame:
    """Read CSV LiDAR data using pandas"""
    # read in CSV data
    df = pd.read_csv(fname, parse_dates=["timestamp"], index_col=0, sep=r",\ ", usecols=["timestamp", field], dtype=float)
   
    # handle nan values (can be 9998 or 9999 - remove anything 9990 or greater)
    df[df[field] > na_value] = np.nan
    
    # return DataFrame
    return df

In [ ]:
# read in a lidar dataset
## New York Bight LiDAR Buoy data
## https://oswbuoysny.resourcepanorama.dnv.com
fname = "datasets/NYSERDA Floating LiDAR Buoy Data/E05_Hudson_South_West_10_min_avg_20220129_20230128.csv"
wind58m_lidar = read_csv_lidar(fname, "lidar_lidar58m_Z10_HorizWS")

### Plot windspeeds

In [ ]:
# plot each timeseries
fig, ax = plt.subplots()
wind50m_reanalysis.plot(ax=ax)
lines = wind58m_lidar.plot(ax=ax).lines

# add x and y limit code here
#plt.gca().set_xlim("2022-04-15", "2022-05-01")
#plt.gca().set_xlim("2022-11-01", "2022-11-15")

plt.legend(lines, ["MERRA2 50m Horiz-WS", "LiDAR 58m Horiz-WS"]);

In [ ]:
from numpy.typing import ArrayLike
def MAE(res: ArrayLike, allow_nan=True) -> ArrayLike:
    """Mean Absolute Error for generic ArrayLike type"""
    
    # define mean function (default numpy mean)
    mean = np.mean
    # use numpy nanmean if allow_nan is True
    if allow_nan:
        mean = np.nanmean
        
    return mean(np.abs(res))

In [ ]:
from numpy.typing import ArrayLike
def RMSE(res: ArrayLike, allow_nan=True) -> ArrayLike:
    """Root Mean Square Error for generic ArrayLike type"""
    
    # define mean function (default numpy mean)
    mean = np.mean
    # use numpy nanmean if allow_nan is True
    if allow_nan:
        mean = np.nanmean
        
    return np.sqrt(mean(np.square(res)))

In [ ]:
lidar_hourly = wind58m_lidar.resample("H").mean()
rea = wind50m_reanalysis.to_series()
diff = lidar_hourly.values - rea.values

print(f"MAE: {MAE(diff):.3f}")
print(f"RMSE: {RMSE(diff):.3f}")

## Interactive Session Part 2: Spatial Data

APIs to access ready made geotiff files on DTU's Global Wind Atlas

See video tutorial: <https://globalwindatlas.info/en/about/VideoTutorials>

In [ ]:
# JABA: examples of APIs to access ready made geotiff files on DTU's Global Wind Atlas
#mmap_name = 'https://globalwindatlas.info/api/gis/country/DNK/wind-speed/100'
#mmap_name = 'https://globalwindatlas.info/api/gis/country/DNK/elevation_w_bathymetry/'
#mmap_name = 'https://globalwindatlas.info/api/gis/country/ITA/wind-speed/100'

## Load datasets

In [ ]:
# JABA: example custom region of interest ("area-1") defined on GWA webpage,
# JABA: and wind speed at different heights above surface made as geostiff data and downloaded
mmap_name10 = "datasets/GWA/area_1_wind-speed_10m.tif"
mmap_name50 = "datasets/GWA/area_1_wind-speed_50m.tif"
mmap_name100 = "datasets/GWA/area_1_wind-speed_100m.tif"
mmap_name150 = "datasets/GWA/area_1_wind-speed_150m.tif"
mmap_name200 = "datasets/GWA/area_1_wind-speed_200m.tif"

## Visualise datasets

In [ ]:
# JABA: 10 m data
fig, ax = plt.subplots()
dataset10 = rasterio.open(mmap_name10)
im = show((dataset10, 1), ax=ax, title='Wind speed at 10 m', vmin=2, vmax=10)
fig.colorbar(im.get_images()[0], ax=ax);

In [ ]:
# JABA: 50 m data
fig, ax = plt.subplots()
dataset50 = rasterio.open(mmap_name50)
im = show((dataset50,1), ax=ax, title='Wind speed at 50 m', vmin=2, vmax=10)
fig.colorbar(im.get_images()[0], ax=ax);

In [ ]:
# JABA: 100 m data
fig, ax = plt.subplots()
dataset100 = rasterio.open(mmap_name100)
im = show((dataset100,1), ax=ax, title='Wind speed at 100 m', vmin=2, vmax=10)
fig.colorbar(im.get_images()[0], ax=ax);

In [ ]:
# JABA: 150 m data
fig, ax = plt.subplots()
dataset150 = rasterio.open(mmap_name150)
im = show((dataset150,1), ax=ax, title='Wind speed at 150 m', vmin=2, vmax=10)
fig.colorbar(im.get_images()[0], ax=ax);

In [ ]:
# JABA: 200 m data
fig, ax = plt.subplots()
dataset200 = rasterio.open(mmap_name200)
im = show((dataset200, 1), ax=ax, title='Wind speed at 200 m', vmin=2, vmax=10)
fig.colorbar(im.get_images()[0], ax=ax);

## Make transect measurement

In [ ]:
# open datasets with xarray
ds10 = xr.open_dataset(mmap_name10, engine='rasterio')
ds50 = xr.open_dataset(mmap_name50, engine='rasterio')
ds100 = xr.open_dataset(mmap_name100, engine='rasterio')
ds150 = xr.open_dataset(mmap_name150, engine='rasterio')
ds200 = xr.open_dataset(mmap_name200, engine='rasterio')

In [ ]:
ds10

In [ ]:
ds10.x.mean(), ds10.y.mean()

In [ ]:
# Make transect at measurement latitude.
windspeed10 = ds10.band_data
windspeed50 = ds50.band_data
windspeed100 = ds100.band_data
windspeed150 = ds150.band_data
windspeed200 = ds200.band_data

# calculate mean of reanlsysis data, this will be used in the transect plot
wind50m_reanalysis_mean=np.mean(wind50m_reanalysis)
#wind50m_reanalysis_mean

# calculate mean of lidar data, this will be used in the transect plot
wind58m_lidar_mean=np.mean(wind58m_lidar)
#wind58m_lidar_mean

 
# JABA: create a transect at the measurement location (lon_meas, lat_meas)
#lon_meas = -74.0 # teest
#lat_meas = 40.0 # test
lon_meas=-73.59
lat_meas=39.48


# JABA: put in the mean value here for the measured and reanalysis data
#data_meas = 6.0
#data_rean = 6.9
data_meas = wind58m_lidar_mean
data_rean = wind50m_reanalysis_mean

 
lat_trans = lat_meas

In [ ]:
# plot 10m windspeed
windspeed10_1d=windspeed10.sel(band=1, y=lat_trans, method='nearest')
windspeed10_1d.plot();

In [ ]:
# add 50m windspeed
windspeed50_1d=windspeed50.sel(band=1, y=lat_trans, method='nearest')

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")

plt.legend();

In [ ]:
# add 100m windspeed
windspeed100_1d=windspeed100.sel(band=1, y=lat_trans, method='nearest')

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")
windspeed100_1d.plot(label="100m")

plt.legend();

In [ ]:
# add 150m windspeed
windspeed150_1d=windspeed150.sel(band=1, y=lat_trans, method='nearest')

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")
windspeed100_1d.plot(label="100m")
windspeed150_1d.plot(label="150m")

plt.legend();

In [ ]:
# add 200m windspeed
windspeed200_1d=windspeed200.sel(band=1, y=lat_trans, method='nearest')

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")
windspeed100_1d.plot(label="100m")
windspeed150_1d.plot(label="150m")
windspeed200_1d.plot(label="200m")

plt.legend();

In [ ]:
# add measurement data
windspeed_meas_1d=xr.DataArray([data_meas], name="band_data",dims={"x"}, coords={"x":[lon_meas]})

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")
windspeed100_1d.plot(label="100m")
windspeed150_1d.plot(label="150m")
windspeed200_1d.plot(label="200m")
windspeed_meas_1d.plot.scatter(label="58m measurement", color="white", marker="x")

plt.legend();

In [ ]:
# add reanalysis data
windspeed_rean_1d=xr.DataArray([data_rean], name="band_data",dims={"x"}, coords={"x":[lon_meas]})

windspeed10_1d.plot(label="10m")
windspeed50_1d.plot(label="50m")
windspeed100_1d.plot(label="100m")
windspeed150_1d.plot(label="150m")
windspeed200_1d.plot(label="200m")
windspeed_meas_1d.plot.scatter(label="58m measurement", color="white", marker="x")
windspeed_rean_1d.plot.scatter(label="50m reanalysis", color="white", marker="o")

plt.legend();

## Learn to drive before taking off in a Ferrari!

<center><img src="images/openclipart_Ferrari-f458-Spider.svg" width="70%"></center>

## Discussion: Next Generation Climate Services



# Should an emphasis be placed on producing data on regular spatio-temporal grids that can easily be processed by end users?

* For example to get extrapolated point data or time series data?
* or should source data be kept “pure” so that end-users “see” any QC issues?

# A vast amount of meteorological and energy generation data is currently proprietary:  is it possible to use this data for the broader good?

* How valuable is this data to the energy transition and hence to energy related climate services?
* Should data owners be forced to provide the data to the broader energy space as a condition of interconnection?
* How do we make it available and protect the interests of the data owner? What might be the role of climate service providers in compilation, QC, and aggregation?

# What are some of the limitations of current NWP/GCM based products used by the energy sector?

* Is the temporal and spatial resolution sufficient? Where might it be a problem?
* Are they sufficiently validated?  The ratio of ground truth to model points is very small.
* Are we too focused on just producing large and long datasets, while not making sure the most important tail events are captured?
    * As a follow up to the above, for instance, NWP does a poor job of stable boundary layers.  What are the ramifications of this for datasets being used in the energy space?
* Is it possible to create products from these incredibly complex and nuanced models, that a non-expert (i.e. non-modeler) user can apply in isolation without potentially reaching erroneous conclusions? How do we make sure users are aware of the limitations?
* How should we “teach” users to apply data from GCM’s to energy space problems. i.e. we know the data has significant uncertainties AND we cannot validate future looking datasets, so how do we make sure users correctly translate uncertainty into their downstream analysis?

# How might new post-processing methods change the quality and quantity of data in the climate services arena?

AI can dramatically reduce the effort of QC’ing or enhancing data but it does so in a fashion that is often untransparent.

* Generative Adversarial Networks (GANs) produce incredible downscaling results that may offer a route to producing physically consistent high-resolution data (spatial and temporal) from existing NWP and GCM data at a fraction of the cost of NWP downscaling. But can we trust it? If the data is largely being used by non-experts, will they find any potential problems before they result in poor decision making?
* Other ML techniques could do QC and fill gaps in data for a fraction of the cost of traditional methods. But again, how do we catch big misses before the become big problems?
* How do we validate the huge datasets that will become generally available and provide that info to end-users to avoid a GIGO situation?
* Is it possible that these techniques could lull us into a sense of security that isn’t real?  Might we end up with massive amounts of data that look physically plausible and appear to better sample the space, but we end up failing to sample scenarios that matter the most. For example, NWP struggles with stable boundary layer.  Might GAN methods simply mask the issue and make it appear that we have data that describes every situation but in reality it sometimes misses the mark where it really matters.

# How might AI methods change how data is provided to users?

* Will we end up with ChatGPT type interfaces where we specify the attributes of the data we need, and a climate service returns the data it deems most appropriate?
* What other changes might AI method bring to how users interface with data?
* What are some of the pros and cons that you see in the nexus of the AI revolution and large climate datasets?

# Wrap up

# Thank you

Thank you kindly to the panellists for their contributions, and to the audience for taking part!

## About these slides

View as `PDF` / `html` or run `ipynb` with instructions at:

<table width=100% height=100% style="font-size: 38px";><tr><td width=30%><a href="https://github.com/jfallon1997/ICEM-2023-ClimateServicesWorkshop"><center><img src="github_qr.png" width="60%"></center></a></td><td width=70%><center><a href="https://linktr.ee/ICEM23ClimateServices">https://linktr.ee/ICEM23ClimateServices</a><br><br><a href="https://github.com/jfallon1997/ICEM-2023-ClimateServicesWorkshop">https://github.com/jfallon1997/<br>ICEM-2023-ClimateServicesWorkshop</a></center></td></tr></table>

[*click here to jump back to the start*](#top)